<a href="https://colab.research.google.com/github/buzhangjiuzhou/DLfinal/blob/master/bert_YaHoo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [55]:
# 导入包
# transformer提供了一些训练好的模型，可以很方便的使用。
!pip install transformers
import tensorflow as tf
import pandas as pd
import os
import tqdm
import random 
import math
# 使用分类的模型，增加了一个head用于分类。
from transformers import BertTokenizer, TFBertForSequenceClassification
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
% matplotlib inline

tf.__version__

'2.2.0'

In [56]:
# 导入现成的模型和分词器
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = TFBertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=10)
model.summary()
model.config

Some weights of the model checkpoint at bert-base-uncased were not used when initializing TFBertForSequenceClassification: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier', 'dropout_151']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model: "tf_bert_for_sequence_classification_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  109482240 
_________________________________________________________________
dropout_151 (Dropout)        multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  7690      
Total params: 109,489,930
Trainable params: 109,489,930
Non-trainable params: 0
_________________________________________________________________


BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5,
    "LABEL_6": 6,
    "LABEL_7": 7,
    "LABEL_8": 8,
    "LABEL_9": 9
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "type_vocab_size": 2,
  "vocab_size": 30522
}


In [57]:
# 数据链接，可以在 https://course.fast.ai/datasets 找到。
# agnews数据集，类别，标题，描述。
ag_url = 'https://s3.amazonaws.com/fast-ai-nlp/yahoo_answers_csv.tgz'

In [58]:
# 下载数据，并指定此时数据集的目录
ag_zip_file = tf.keras.utils.get_file(origin=ag_url,fname='yahoo_answers_csv.tgz', extract=True)
base_dir = os.path.join(os.path.dirname(ag_zip_file), 'yahoo_answers_csv')
os.listdir(base_dir)

['classes.txt', 'readme.txt', 'test.csv', 'train.csv']

In [59]:
# 读取数据
# base_dir = ''
train = pd.read_csv(os.path.join(base_dir, 'train.csv'), header=None)
test = pd.read_csv(os.path.join(base_dir, 'test.csv'), header=None)
print(train)
print(len(train))
print(train.head())
train = train.dropna()
test = test.dropna()
train = train.sample(n=50000)
print(len(train))
print(len(test))
print(test.head())
f = open(os.path.join(base_dir, 'classes.txt'))
classes = f.readlines()
classes = [s.strip() for s in classes]
print(classes)

         0  ...                                                  3
0        5  ...  Optical mice use an LED and a camera to rapidl...
1        6  ...  i hear that the mojave road is amazing!<br />\...
2        3  ...  Trans fats occur in manufactured foods during ...
3        7  ...  according to the www.fedex.com web site:\nAir ...
4        7  ...  renting vs buying depends on your goals. <br /...
...     ..  ...                                                ...
1399995  3  ...  I increased my height 2 feet afterwards so yes...
1399996  7  ...  ebay, electronic boutique, babbages or flea ma...
1399997  3  ...  It can be normal as long as they are not cance...
1399998  1  ...                  Main hindi bol sakti hoon.kahiye.
1399999  5  ...  ★★ HELP WITH SEARCHING ★★\n\n◙ I used to have ...

[1400000 rows x 4 columns]
1400000
   0  ...                                                  3
0  5  ...  Optical mice use an LED and a camera to rapidl...
1  6  ...  i hear that the mojave road

In [60]:
# transformers自带的tokenizer中的encoder会把一段文本进行编码，然后增加上CLS和SEP，其中CLS的id是101，SEP的编码是102,PAD是0。
# 所以  a   dog   is  not   a   table
# [cls]  a   dog   is  not   a   table  [sep]
# 101   1037  3899  2003 2025  1037  2795   102
# %pprint #让列表横过来，好看一些。
tokenizer.encode(text='a dog is not a table', padding='max_length',max_length=512)[:20]

[101, 1037, 3899, 2003, 2025, 1037, 2795, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [61]:
train.head()

,0,1,2,3
243629,7,credit check reqiuring no credit card?,trying to find site to get credit report witho...,Free annual credit check.com
1205041,9,looking for info on work @home jobs?,"Mom of 3 very interested @woring at home,don't...",http://makemoney.yacam.altervista.org here you...
1083217,8,I need help Finding these songs for FREE to do...,1. Do Yo Dance-Ben Hill Squad\n2.Ain't No *itc...,Try these links below-\n\nhttp://www.seekasong...
1391352,1,I am a spiritual person.My boyfriend and I go ...,"to become closer to God.I pray every morning, ...",Being spiritual doesn't make you a good person...
1004903,7,"is there an Ulmerton Investment Itd, in the UK?","Ulmerton Inv. ltd, Roxwell Trading Park UK. ...",Companies house says they have been dissolved!...


In [62]:
# 预处理数据
# 按照上面的例子把数据集中的文本进行处理，并且得到对应的labels。
max_length = 512

# 把太长的句子给截断，防止后面出问题。
max_length_temp = math.floor(max_length * 0.9)
for i in tqdm.notebook.trange(train.shape[0]):
  if len(train[3][train[3].index[i]]) > max_length_temp:
    train[3][train[3].index[i]] = train[3][train[3].index[i]][0 : max_length_temp]
for i in tqdm.notebook.trange(test.shape[0]):
  if len(test[3][test[3].index[i]]) > max_length_temp:
    test[3][test[3].index[i]] = test[3][test[3].index[i]][0 : max_length_temp]

train_ids = [tokenizer.encode(text=sent, padding='max_length', max_length=max_length, return_tensors="tf") for sent in tqdm.notebook.tqdm(train[3])]
test_ids = [tokenizer.encode(text=sent, padding='max_length',max_length=max_length, return_tensors="tf") for sent in tqdm.notebook.tqdm(test[3])]
train_labels = train[0].values - 1
test_labels = test[0].values - 1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


In [63]:
# 把数据转成tensorflow张量
# train_ids是tf.Tensor组成得列表，所以用concat组合一下就行
train_ids = tf.concat(train_ids, 0)
# 把train_mask初始化为1，然后把train_ids等于0（PAD的部分）对应的值赋为0
train_mask = tf.ones(train_ids.shape)
train_mask = tf.where(tf.math.greater(train_ids, 0), train_mask, 0)
# labels本身是numpy数组，转为tf.Tensor
train_labels = tf.convert_to_tensor(train_labels)

# 测试集的处理同理test
test_ids = tf.concat(test_ids, 0)
test_mask = tf.ones(test_ids.shape)
test_mask = tf.where(tf.math.greater(test_ids, 0), test_mask, 0)
test_labels = tf.convert_to_tensor(test_labels)

print(train_ids[0])
print(train_mask[0])
print(train_labels[0])

tf.Tensor(
[ 101 2489 3296 4923 4638 1012 4012  102    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0 

In [64]:
# 训练参数
epochs = 2
batch_size = 4
validation_rate = 0.1

In [65]:
# 训练
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

In [66]:
# 模型训练
model.fit(x=[train_ids, train_mask], 
     y=train_labels, 
     batch_size=batch_size, 
     epochs=epochs, 
     verbose=1, 
     callbacks=None,
     validation_split=validation_rate, 
     validation_data=None, 
     shuffle=True)

Epoch 1/2
11250/11250 [==============================] - 3440s 306ms/step - loss: 1.2352 - accuracy: 0.6110 - val_loss: 1.1490 - val_accuracy: 0.6440
Epoch 2/2
11250/11250 [==============================] - 3439s 306ms/step - loss: 0.9483 - accuracy: 0.7060 - val_loss: 1.2136 - val_accuracy: 0.6288


In [67]:
# 模型测试
model.evaluate(x=[test_ids, test_mask],
        y=test_labels, 
        batch_size=4, 
        verbose=1)

8067/8067 [==============================] - 883s 109ms/step - loss: 1.1956 - accuracy: 0.6344


[1.1956062316894531, 0.6343716382980347]